In [ ]:
def load_patient():
    import os
    import slicer
    from google.cloud import storage

    #Edit this path to the location of the Google Storage credentials file
    key_path = 'C:/Users/Nicolas/Downloads/zinc-citron-387817-70b815932bac.json'

    # Create the storage client using the service account key file and establish bucket name
    client = storage.Client.from_service_account_json(key_path)
    bucket_name = 'rice_d2k_biocv'
    bucket = client.get_bucket(bucket_name)

    # Prompt the user for an integer
    patient_num = input("Enter the patient number (as an integer): ")

    # Pad the integer with leading zeros to total 5 digits
    patient_num_str = str(patient_num).zfill(5)

    # Create a folder for the patient in the Subject Hierarchy
    shNode = slicer.vtkMRMLSubjectHierarchyNode.GetSubjectHierarchyNode(slicer.mrmlScene)
    parent_folder_id = shNode.CreateFolderItem(shNode.GetSceneItemID(), f"UPENN-GBM-{patient_num_str}")

    # Define the volume sets and corresponding subfolder names
    volume_sets = [
        {
            "folder_name": "images_annot_reduced",
            "file_types": ["FLAIR_cut", "T1GD_cut", "T1_cut", "T2_cut"]
        },
        {
            "folder_name": "images_annot_reduced_norm",
            "file_types": ["FLAIR", "T1", "T1GD", "T2"]
        },
        {
            "folder_name": "images_structural",
            "file_types": ["FLAIR", "T1", "T1GD", "T2"]
        }
    ]

    # Load volumes from each volume set
    missing_volumes = []
    for volume_set in volume_sets:
        folder_name = volume_set["folder_name"]
        file_types = volume_set["file_types"]
        volume_set_folder_id = shNode.CreateFolderItem(parent_folder_id, folder_name)

        for file_type in file_types:
            if folder_name == "images_structural":
                volume_filename = f"content/data/{folder_name}/UPENN-GBM-{patient_num_str}_11/UPENN-GBM-{patient_num_str}_11_{file_type}.nii.gz"
            else:
                volume_filename = f"content/data/{folder_name}/UPENN-GBM-{patient_num_str}_11_{file_type}.nii.gz"

            save_path = os.path.join(slicer.app.temporaryPath, os.path.basename(f"{folder_name}_{file_type}.nii.gz"))
            blob = bucket.blob(volume_filename)

            try:
                blob.download_to_filename(save_path)
                volume = slicer.util.loadVolume(save_path)
                if volume:
                    shNode.SetItemParent(shNode.GetItemByDataNode(volume), volume_set_folder_id)
                else:
                    print(f"Failed to load {file_type} volume from {volume_filename}")
            except:
                missing_volumes.append(volume_filename)

    # Define the segmentation file paths
    segmentation_folder_id = shNode.CreateFolderItem(parent_folder_id, "Segmentations")
    segmentation_filenames = [
        f"content/data/automated_segm/UPENN-GBM-{patient_num_str}_11_automated_approx_segm.nii.gz",
        f"content/data/automated_segm_reduced/UPENN-GBM-{patient_num_str}_11_automated_approx_segm_cut.nii.gz",
        f"content/data/images_annot_reduced/UPENN-GBM-{patient_num_str}_11_segm_cut.nii.gz",
        f"content/data/images_annot_reduced_norm/UPENN-GBM-{patient_num_str}_11_segm.nii.gz",
        f"content/data/images_segm/UPENN-GBM-{patient_num_str}_11_segm.nii.gz"
    ]

    # Load the segmentation files into 3D Slicer
    missing_segmentations = []
    for segmentation_filename in segmentation_filenames:
        save_path = os.path.join(slicer.app.temporaryPath, os.path.basename(segmentation_filename.replace("/", "_")))
        blob = bucket.blob(segmentation_filename)

        try:
            blob.download_to_filename(save_path)
            segmentation = slicer.util.loadSegmentation(save_path)
            if segmentation:
                shNode.SetItemParent(shNode.GetItemByDataNode(segmentation), segmentation_folder_id)
            else:
                print(f"Failed to load segmentation from {segmentation_filename}")
        except:
            missing_segmentations.append(segmentation_filename)

    # Cleanup: Remove the downloaded files
    for volume_filename in missing_volumes + segmentation_filenames:
        save_path = os.path.join(slicer.app.temporaryPath, os.path.basename(volume_filename.replace("/", "_")))
        os.remove(save_path)

    # Print missing files
    if missing_volumes:
        print("The following volumes were not found on Google Storage:")
        for volume_filename in missing_volumes:
            print(volume_filename)

    if missing_segmentations:
        print("The following segmentations were not found on Google Storage:")
        for segmentation_filename in missing_segmentations:
            print(segmentation_filename)

    #Defaults volume and segment visible in slicer
    # Find the nodes of the loaded volumes and segmentations
    volume_nodes = slicer.util.getNodesByClass("vtkMRMLScalarVolumeNode")
    segmentation_nodes = slicer.util.getNodesByClass("vtkMRMLSegmentationNode")

    # Find the desired volume node by name
    desired_volume_name = "images_annot_reduced_norm_T1"
    desired_volume_node = None
    for volume_node in volume_nodes:
        if volume_node.GetName() == desired_volume_name:
            desired_volume_node = volume_node
            break

    # Set visibility for the desired volume and segmentation
    slicer.util.setSliceViewerLayers(background=desired_volume_node)

    # Find the desired segmentation node by name
    desired_segmentation_name = f"content/data/images_annot_reduced_norm/UPENN-GBM-{patient_num_str}_11_segm.nii.gz".replace("/", "_")
    desired_segmentation_node = None
    for segmentation_node in segmentation_nodes:
        if segmentation_node.GetName() == desired_segmentation_name:
            desired_segmentation_node = segmentation_node
            break

    # Set visibility for the desired segmentation
    for segmentation_node in segmentation_nodes:
        if segmentation_node != desired_segmentation_node:
            segmentation_node.SetDisplayVisibility(False)

    if desired_segmentation_node:
        desired_segmentation_node.SetDisplayVisibility(True)

In [ ]:
def load_dataset():
    import os
    import slicer
    from google.cloud import storage

    # Edit this path to the location of the Google Storage credentials file
    key_path = 'C:/Users/Nicolas/Downloads/zinc-citron-387817-70b815932bac.json'

    # Create the storage client using the service account key file and establish bucket name
    client = storage.Client.from_service_account_json(key_path)
    bucket_name = 'rice_d2k_biocv'
    bucket = client.get_bucket(bucket_name)

    # Prefix for data folder in the bucket
    prefix = 'content/data/'

    # Set the delimiter
    delimiter = '/'

    # Create a list to hold the unique subfolder names
    subfolders = []

    # Use the delimiter to simulate a directory structure
    blobs = bucket.list_blobs(prefix=prefix, delimiter=delimiter)

    # Get the prefixes (subdirectories)
    for page in blobs.pages:
        subfolders.extend(page.prefixes)

    # Display the subfolders
    print("Subfolders within 'content/data/':")
    for i, subfolder in enumerate(subfolders, start=1):
        # Remove the prefix and trailing slash, display with a preceding number
        print(f"{i}. {subfolder[len(prefix):-len(delimiter)]}")

    # Prompt the user to select a subfolder by entering a number
    while True:
        subfolder_number = input("Enter the number of the subfolder you want to load: ")
        if subfolder_number.isdigit() and 1 <= int(subfolder_number) <= len(subfolders):
            break
        else:
            print("Invalid input. Please enter a valid number for a subfolder.")

    # Get the selected subfolder name
    subfolder_name = subfolders[int(subfolder_number) - 1][len(prefix):-len(delimiter)]

    # Check if the subfolder exists within the 'data/' folder
    prefix = 'data/' + subfolder_name + '/'
    blobs = list(bucket.list_blobs(prefix=prefix))

    if any(blobs):
        print(f"The subfolder '{subfolder_name}' exists within the 'data/' folder.")
    else:
        print(f"The subfolder '{subfolder_name}' does not exist within the 'data/' folder.")
        return

    # Create a folder for the subfolder in the Subject Hierarchy
    shNode = slicer.vtkMRMLSubjectHierarchyNode.GetSubjectHierarchyNode(slicer.mrmlScene)
    parent_folder_id = shNode.CreateFolderItem(shNode.GetSceneItemID(), f"{subfolder_name}")

    # Prompt the user to select which file types to load
    load_t1 = input("Do you want to load T1 volumes? (y/n): ").lower() == "y"
    load_t1gd = input("Do you want to load T1GD volumes? (y/n): ").lower() == "y"
    load_t2 = input("Do you want to load T2 volumes? (y/n): ").lower() == "y"
    load_flair = input("Do you want to load FLAIR volumes? (y/n): ").lower() == "y"
    load_segm = input("Do you want to load Segmentations? (y/n): ").lower() == "y"

    # Progress bar
    progressBar = slicer.util.createProgressDialog()
    progressBar.minimum = 0
    progressBar.maximum = len(blobs)
    progress = 0

    # Iterate through the blobs and load volumes
    for blob in blobs:
        if blob.name.endswith('.gz'):
            volume_filename = os.path.basename(blob.name)
            volume_type = volume_filename.split("_")[2].split(".")[0]

            if (
                (load_t1 and volume_type == "T1") or
                (load_t1gd and volume_type == "T1GD") or
                (load_t2 and volume_type == "T2") or
                (load_flair and volume_type == "FLAIR")
            ):
                save_path = os.path.join(slicer.app.temporaryPath, volume_filename)
                blob.download_to_filename(save_path)
                volume = slicer.util.loadVolume(save_path)
                if volume:
                    shNode.SetItemParent(shNode.GetItemByDataNode(volume), parent_folder_id)
                else:
                    print(f"Failed to load volume: {volume_filename}")
            elif load_segm and "segm" in volume_type:
                save_path = os.path.join(slicer.app.temporaryPath, volume_filename)
                blob.download_to_filename(save_path)
                segmentation = slicer.util.loadSegmentation(save_path)
                if segmentation:
                    shNode.SetItemParent(shNode.GetItemByDataNode(segmentation), parent_folder_id)
                    segmentation.GetDisplayNode().SetVisibility(False)

                else:
                    print(f"Failed to load segmentation: {volume_filename}")

        # Update progress bar
        progress += 1
        progressBar.value = progress
        slicer.app.processEvents()

    progressBar.close()

    print("Volume loading completed.")

    # Get the Data module widget
    data_widget = slicer.modules.data.widgetRepresentation()

    # Find and collapse the subject hierarchy tree
    shTreeView = data_widget.findChild('QTreeView')
    shTreeView.collapseAll()